# Baseline Models 

This method only works for sentiment transfer. This is because the attribute markers in semantic transfer are less nuanced than those needed for formality transfer. The attribute markers used here will not learn words that are more formal and cannot be generalized as easily. 

These are the model proposed to compare against DeleteOnly and DeleteAndRetrieve

## Here we can see that this will not work

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from nltk import ngrams

import os
import re

## Load Data

In [5]:
formal = open('../Data/Unsupervised Data/Entertainment_Music/U_Formal_EM_Test.txt').read()
informal = open('../Data/Unsupervised Data/Entertainment_Music/U_Informal_EM_Val.txt').read()

# formal_holdout = open('../../Data/Unsupervised Data/Entertainment_Music/U_Formal_EM_ValTest.txt').read()
# informal_holdout = open('../../Data/Unsupervised Data/Entertainment_Music/U_Informal_EM_ValTest.txt').read()

In [6]:
def process_sequence(seq):
    """This inserts a space in between the last word and a period"""
    s = re.sub('([.,!?()])', r' \1 ', seq)
    s = re.sub('\s{2,}', ' ', s)
    return s

f_corpus = [process_sequence(seq) for seq in formal.split('\n')]
if_corpus = [process_sequence(seq) for seq in informal.split('\n')]

In [7]:
print("{} formal sequences and {} informal sequences for evaluation.".format(len(f_corpus), 
                                                                             len(if_corpus)))

1083 formal sequences and 2878 informal sequences for evaluation.


## Static Variables

This is an incredibly trivial approach to implementing unsupervised style transfer. The process is to identify a sequence $x^{tgt}$ that is similar in content to $x^{src}$ and return that sequence.

We define an attribute marker $u$ if $s(u,v) \geq \gamma$, where $\gamma$ is a specified tolerance and 
$$ s(u,v) = \frac{\text{count}(u, D_v) + \lambda}{(\sum_{v^\prime \in V, v^\prime \neq v} \text{count}(u, D_{v^\prime })) + \lambda} $$
where $D_v$ 

In [8]:
def get_ngrams(corpus, n):
    """
    Gets all n grams less than or equal to n
    """
    return {' '.join(gram):0 for seq in corpus for l in range(2,n) 
        for gram in ngrams(seq.split(), l)
    }

def get_ngram_count(corpus, n):
    """Get all ngrams with counts"""
    # get ngrams
    n_grams = get_ngrams(corpus, n)

    # count!
    for seq in corpus:
        for gram in n_grams:
            if gram in seq:
                n_grams[gram] += 1

    return n_grams 

In [18]:
    def jaccard_similarity(seq_a, seq_b):
        a = set(seq_a.split('\n'))
        b = set(seq_b.split('\n'))
        c = a.intersection(b),
        return float(len(c) / (len(a) + len(b) - len(c)))
    
    def compute_similarities(inf, f):
        evaluations = {}
        for seq_if in inf:
            best = -1
            for seq_f in f:
                score = jaccard_similarity(seq_if, seq_f)
                if score > best:
                    best = score
                    best_if = seq_if
                    best_f = seq_f
            evaluations[best_if] = best_f
        return evaluations
    
    def see_match(seq_a, target_dict):
        seq_b = target_dict[seq_a]
        sim = jaccard_similarity(seq_a, seq_b)
        print("Informal Seq: {} | Formal Seq: {} | Jaccard Similarity {}".format(seq_a,
                                                                                 seq_b,
                                                                                 sim))

In [9]:
if_count = get_ngram_count(if_corpus, 5)
f_count = get_ngram_count(f_corpus, 5)

In [11]:
max(f_count, key=f_count.get)

'e .'

In [12]:
f_count['e .']

178

In [15]:
max(if_count, key=if_count.get)

'. .'

In [17]:
if_count['. .']

441

Clearly this method is not worth pursuing for formality transfer, but does imply the rule-assisted network has great potential!